In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer


/home/abhinav/miniconda3/envs/inference/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

# Load the dataset
dataset = load_dataset("verifiers-for-code/tester3", split='train')

# Load the tokenizer for the chat template
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

def apply_template(example):
    chat_prompt = tokenizer.apply_chat_template(
        [
            {
                "role": "system",
                "content": "You are given the start of a function for a Python program. Your job is to produce a detailed plan. First, analyze and think, by using <thinking> XML tags, and then, produce a plan, in <plan> tags.",
            },
            {"role": "user", "content": example['input']},
            {"role": "assistant", "content": example['structured_plans_Llama-3_1-70B']},
        ],
        tokenize=False
    )
    return {"text": chat_prompt}

/home/abhinav/miniconda3/envs/inference/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
processed_dataset = dataset.map(apply_template)

# Keep only the 'text' column
processed_dataset = processed_dataset.remove_columns([col for col in processed_dataset.column_names if col != 'text'])


Map: 100%|██████████| 27224/27224 [00:05<00:00, 4938.72 examples/s]


In [4]:
processed_dataset[0]['text']

'<|system|>\nYou are given the start of a function for a Python program. Your job is to produce a detailed plan. First, analyze and think, by using <thinking> XML tags, and then, produce a plan, in <plan> tags.<|end|>\n<|user|>\ndef add_numbers(num1: int | float, num2: int | float) -> int | float:\n    """\n    Return the sum of two numbers.\n    \n    >>> add_numbers(5, 3)\n    8\n    >>> add_numbers(-5, 3)\n    -2\n    >>> add_numbers(5.5, 3.2)\n    8.7\n    """<|end|>\n<|assistant|>\n<thinking>\nThe problem at hand is to implement a simple function `add_numbers` that takes two numbers as input and returns their sum. The input numbers can be either integers or floating-point numbers. Let\'s break it down:\n\n1. Main objective: The function should calculate the sum of two numbers.\n2. Key considerations:\n   a. Input types: Both integer and floating-point numbers are allowed.\n   b. Output type: The result should be of the same type as the inputs (integer or float).\n3. Key steps:\n  

In [5]:
processed_dataset.push_to_hub('verifiers-for-code/structured_ft_27k_phi3')

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Uploading the dataset shards: 100%|██████████| 1/1 [00:01<00:00,  1.64s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/verifiers-for-code/structured_ft_27k_phi3/commit/37dc38983b9a43f5972bdd03f1ad7cb79124f373', commit_message='Upload dataset', commit_description='', oid='37dc38983b9a43f5972bdd03f1ad7cb79124f373', pr_url=None, pr_revision=None, pr_num=None)